# 📘 Bookflix – Sprint 4: Sistema de Analisis de Sentimientos
**Autores:**  
- Daniel Fabián Rodríguez Lorenzo  
- Tatiana María Quintas Rodríguez  
- Miguel Ángel Seara Losada  
- David Simón Nóvoa  

**Objetivo:**  
Implementar un sistema de analisis de sentimientos para clasificar reseñas en positivas o negativas.  

# **Análisis y Predicción de Ratings de Libros**

En este notebook realizaremos un análisis completo del dataset Books_rating_5000.csv, el cual contiene información sobre miles de libros (título, autor, género, páginas, etc.) junto con sus valoraciones numéricas.
El objetivo será explorar los datos, preprocesarlos y finalmente construir un modelo supervisado capaz de predecir la valoración (rating) de un libro a partir de sus características.

El propósito es entender paso a paso cómo abordar un flujo de trabajo completo en un proyecto de machine learning con datos reales.

# **Introducción**

Para nuestro sistema de valoración, usaremos un algoritmo de clasificación supervisado. Para entrenar y probar nuestro algoritmo, usaremos 2 datasets preparados anteriormente : training_data.xlsx y testing_data.xlsx, que son distintos datasets pero que tienen la misma estructura. Usaremos las columnas review y label, siendo la primera el texto a valorar y la segunda una etiqueta que indica al algoritmo si el texto es positivo (1) o negativo (0).
Para ambos datasets, inicialmente realizaremos un preprocesado de los textos para que sea mas facil trabajar con ellos, aplicando la _tokenización_, la eliminación de _stopwords_ y la _stemmización_ de los tokens. Los textos obtenidos del preprocesado se representarán mediante una Bag Of Words (BoW).

# **Cargar datos de entrenamiento**

Inicialmente cargamos el dataset training_data.xlsx, que contiene información de aproximadamente 1200 reseñas de libros.

(Nota : Primero es necesario cargar manualmente el archivo al notebook)

Mostramos una vista previa de los datos para comprobar que se han cargado correctamente, y comprobamos cuantos elementos hay positivos y negativos (1 = positivo, 0 = negativo).

In [33]:
import pandas as pd

trainingData = pd.read_excel('training_data.xlsx')
#Usaremos solo 1000 reseñas
trainingData = trainingData.head(1000)
trainingData

,review,label
0,I can't remember the last time I read such a d...,0
1,This is the first book I've read by Clive Cuss...,0
2,The book was kind of boring. I didn't even fin...,0
3,"Suzanne Chazin's debut novel, ""The Fourth Ange...",0
4,Hi! my name is Roy Chan and I am reading this ...,0
...,...,...
995,Seasons of the Elk is an excellent resource fo...,1
996,"After I start to photograph wildlife, especial...",1
997,This is the best textbook I have ever read and...,1
998,I use this text for my English 202 Class on Sa...,1


In [34]:
trainingData['label'].value_counts()

,count
label,
0,541
1,459


# **Preprocesamiento de los datos**

En esta celda se preprocesa el texto almacenado en el campo "review" y se almacena en un nuevo DataFrame con nombre "preprocessedData", que contendra la información del DataFrame original más una columna "preprocessedText".
El preprocesamiento incluye los siguientes pasos:
 - *Tokenización* del texto (mediante la libreria *nltk*) : Se divide el texto en palabras (tokens), de forma que se obtiene un array compuesto por las palabras que constituyen del texto, separandolas mediante  los delimitadores habituales ( espacios, comas, puntos...).

 - Eliminación de *stopwords* (usando de referencia las stopwords por defecto de la libreria *nltk*) : Consiste en eliminar palabras que no aportan significado al texto (stopwords), como pueden ser articulos, pronombres y preposiciones. Las stopwords a eliminar son dependientes del idioma, por lo que el dataset usado debe estar en Ingles.

- *Stemmizacion del texto* (usando el algoritmo *PorterStemmer*) : Se eliminan variaciones de palabras provocadas por conjugación de verbos y por uso de plurales y generos. Para conseguirlo, se obtiene la raiz semantica de todas las palabras.

Al finalizar el preprocesamiento, se muestran los datos para verificar que el proceso se ha realizado con exito.

In [35]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Added to resolve a LookupError


ps = PorterStemmer()

preprocessedText  = []

for row in trainingData.itertuples():
    text = word_tokenize(row[1]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)

    preprocessedText.append(text)

preprocessedData = trainingData
preprocessedData['preprocessedText'] = preprocessedText

preprocessedData

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,review,label,preprocessedText
0,I can't remember the last time I read such a d...,0,i ca rememb last time i read dire book i howl ...
1,This is the first book I've read by Clive Cuss...,0,thi first book i read clive cussler i doubt i ...
2,The book was kind of boring. I didn't even fin...,0,the book kind bore i even finish read i also t...
3,"Suzanne Chazin's debut novel, ""The Fourth Ange...",0,suzann chazin debut novel the fourth angel fol...
4,Hi! my name is Roy Chan and I am reading this ...,0,hi name roy chan i read i 7th grade i decid re...
...,...,...,...
995,Seasons of the Elk is an excellent resource fo...,1,season elk excel resourc anyon interest studi ...
996,"After I start to photograph wildlife, especial...",1,after i start photograph wildlif especi larg m...
997,This is the best textbook I have ever read and...,1,thi best textbook i ever read way i could unde...
998,I use this text for my English 202 Class on Sa...,1,i use text english 202 class saipan mixtur cha...


# Creación del Bag of Words

Una vez se ha preprocesado las reseñas de los libros, se representarán los textos obtenidos como Bag of Words (BoW).


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['preprocessedText'])
textsBoW= bagOfWordsModel.transform(preprocessedData['preprocessedText'])
textsBoW.shape


(1000, 8332)

## **Entrenamiento de un algoritmo de clasificación**

Emplearemos SVM (Support Vector Machine), que es un algoritmo de aprendizaje automatico supervisado.

In [37]:
from sklearn import svm
svc = svm.SVC(kernel='linear') #Modelo de clasificación

X_train = textsBoW #Documentos
Y_train = trainingData['label'] #Etiquetas de los documentos
svc.fit(X_train, Y_train) #Entrenamiento

SVC(kernel='linear')

## **Evalución del algoritmo de clasificación**

Usaremos
Inicialmente el dataset testing_data.xlsx, que contiene información de 1000 reseñas de libros.

(Nota : Primero es necesario cargar manualmente el archivo al notebook)

Mostramos una vista previa de los datos para comprobar que se han cargado correctamente, y comprobamos cuantos elementos hay positivos y negativos (1 = positivo, 0 = negativo).

In [38]:
testingData = pd.read_excel('testing_data.xlsx')
testingData

,review,label
0,It was just not what i thought it would be for...,0
1,I bought this book and took the test but it wa...,0
2,How dare you Chambers try to profit off Iraq. ...,0
3,Your driving a 68ft truck with 30 tons of carg...,0
4,You are a liar and never should have made this...,0
...,...,...
995,This small book of sonnets is a joy to read. T...,1
996,"This was a Valentines Day gift for my husband,...",1
997,Ive never read a book like this befor. Ok...so...,1
998,"At the ages of 7-9, I was a very sick little g...",1


In [39]:
testingData['label'].value_counts()

,count
label,
1,552
0,448


## **Procesado de los datos de Evaluación**
Se realiza el mismo preprocesado que se realizo sobre los datos de entrenamiento, obteniendo un Bag of Words (debe usarse la misma representación que con los datos de entrenamiento).

In [40]:
ps = PorterStemmer()

preprocessedTestingText  = []

for row in testingData.itertuples():
    text = word_tokenize(row[1]) ## indice de la columna que contiene el texto
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)

    preprocessedTestingText.append(text)

preprocessedTestData = testingData
preprocessedTestData['preprocessedText'] = preprocessedTestingText

textsBoWTest= bagOfWordsModel.transform(preprocessedTestData['preprocessedText'])
textsBoWTest.shape

(1000, 8332)

## **Obtención y Evaluación de las predicciones**
Inicialmente se obtienen las predicciones y se almacenan en el array predictions, y a continuación se evaluan distintas metricas, como pueden ser la precision (cuantas predicciones fueron correctas, es decir, mide cuantos falsos positivos ocurren), la sensibilidad o recall (cuantos elementos del tipo X se han conseguido predecir, es decir, mide cuantos falsos negativos ocurren), o la f1-score (evaluación conjunta de precisión y sensibilidad).

In [41]:
from sklearn.metrics import classification_report

X_test = textsBoWTest #Reviews de testing a evaluar
#Obtención y almacenamiento de las predicciones del clasificador
predictions = svc.predict(X_test)


Y_test = testingData['label'] #Etiquetas reales de los documentos
print (classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.69      0.85      0.76       448
           1       0.85      0.69      0.76       552

    accuracy                           0.76      1000
   macro avg       0.77      0.77      0.76      1000
weighted avg       0.78      0.76      0.76      1000

